# Цели
Здесь я собираюсь сделать модель для предсказания цены закрытия акции по информации за $N$ предыдущих дней. 
А так же проверить следущие гипотезы:
1. Рынок приспосабливается к стратегиям. Иными словами, чем дальше период на котором обучалась модель от цели предсказания, тем ниже точность предсказания.
2. Внешние данны важны. Иными словами, предсказания котировок с ипользованием внешних текстов получаются точнее, чем без них.

В роли внешних текстов используются [новости с MOEX](https://www.moex.com/ru/news/?ncat=106). А точнее, их заголовки.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer

# Обработка данных   
## Исходные данные
Здесь мы собираем данные как есть

### Акции

In [2]:
path_to_tools = 'data/tools/list_tools.txt'
df_tools = pd.read_csv(path_to_tools,sep='\t')
df_tools.head()

,SECID,BOARDID,SHORTNAME,PREVPRICE,LOTSIZE,FACEVALUE,STATUS,BOARDNAME,DECIMALS,SECNAME,...,PREVDATE,ISSUESIZE,ISIN,LATNAME,REGNUMBER,PREVLEGALCLOSEPRICE,CURRENCYID,SECTYPE,LISTLEVEL,SETTLEDATE
0,ABIO,TQBR,iАРТГЕН ао,73.50,10,0.10,A,Т+: Акции и ДР - безадрес.,2,"ПАО ""Артген""",...,2024-12-12,92645451,RU000A0JNAB6,ARTGEN ao,1-01-08902-A,74.420,SUR,1,2,2024-12-16
1,ABRD,TQBR,АбрауДюрсо,165.00,10,1.00,A,Т+: Акции и ДР - безадрес.,1,Абрау-Дюрсо ПАО ао,...,2024-12-12,98000184,RU000A0JS5T7,Abrau-Durso ao,1-02-12500-A,165.000,SUR,1,3,2024-12-16
2,ACKO,TQBR,АСКО ао,3.58,100,1.00,A,Т+: Акции и ДР - безадрес.,2,АСКО ПАО ао,...,2024-12-12,536000000,RU000A0JXS91,ASKO ao,1-01-52065-Z,3.580,SUR,1,3,2024-12-16
3,AFKS,TQBR,Система ао,11.93,100,0.09,A,Т+: Акции и ДР - безадрес.,3,"АФК ""Система"" ПАО ао",...,2024-12-12,9650000000,RU000A0DQZE3,AFK Sistema,1-05-01669-A,11.932,SUR,1,1,2024-12-16
4,AFLT,TQBR,Аэрофлот,48.54,10,1.00,A,Т+: Акции и ДР - безадрес.,2,Аэрофлот-росс.авиалин(ПАО)ао,...,2024-12-12,3975771215,RU0009062285,Aeroflot,1-01-00010-A,48.550,SUR,1,1,2024-12-16



### Свечи

In [3]:
path_to_all_tools_candles = 'data/candles'
dfs = []
for path_to_candles in Path(path_to_all_tools_candles).glob('*'):
    df_candles = pd.read_csv(path_to_candles,sep='\t')
    df_candles['SECID'] = path_to_candles.name[:-6]
    dfs.append(df_candles)
df_candles = pd.concat(dfs)
df_candles.head()

C:\Users\user\AppData\Local\Temp\ipykernel_22316\2080516023.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_candles = pd.concat(dfs)


,begin,end,open,high,low,close,value,volume,SECID
0,2014-06-09 00:00:00,2014-06-09 23:59:59,15.508,15.508,14.801,15.000,22579.4,1500.0,ABIO
1,2014-06-10 00:00:00,2014-06-10 23:59:59,14.871,15.153,14.871,14.881,25469.9,1700.0,ABIO
2,2014-06-11 00:00:00,2014-06-11 23:59:59,15.000,15.000,14.999,14.999,2999.9,200.0,ABIO
3,2014-06-16 00:00:00,2014-06-16 23:59:59,15.468,15.468,15.000,15.000,24522.3,1600.0,ABIO
4,2014-06-17 00:00:00,2014-06-17 23:59:59,15.100,15.120,15.100,15.120,738392.0,48900.0,ABIO



### Текст

In [4]:
path_to_text = 'data/texts/moex_title.txt'
df_text = pd.read_csv(path_to_text)
df_text.head()

,id,tag,title,published_at,modified_at
0,75803,site,"О внесении изменений в Список ценных бумаг, до...",2024-12-13 17:34:03,2024-12-13 14:34:05
1,75802,site,"Цена исполнения фьючерсов на сахар, декабрь 2024",2024-12-13 17:30:45,2024-12-13 17:30:45
2,75801,site,Дополнительные условия проведения торгов отдел...,2024-12-13 17:25:56,2024-12-13 17:25:56
3,75799,site,"13.12.2024, 17-24 (мск) изменены значения нижн...",2024-12-13 17:24:00,2024-12-13 17:25:10
4,75797,site,"13.12.2024, 17-16 (мск) изменены значения верх...",2024-12-13 17:16:09,2024-12-13 17:20:04


## Входные данные
Здесь мы приводим данные к универсальному виду, который и будем в дальнейшем передавать в модель для дальнейшей предобработки

### Акционные данные

In [5]:
df_tools = df_tools[['SECID','BOARDID','SHORTNAME','SECNAME','LOTSIZE','SECTYPE','LISTLEVEL']]
df_tools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SECID      260 non-null    object
 1   BOARDID    260 non-null    object
 2   SHORTNAME  260 non-null    object
 3   SECNAME    260 non-null    object
 4   LOTSIZE    260 non-null    int64 
 5   SECTYPE    260 non-null    object
 6   LISTLEVEL  260 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 14.3+ KB


In [6]:
df_candles['day_date'] = df_candles['begin'].map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d')) 
df_candles = df_candles[['SECID','day_date','close','value','volume']]
df_candles.head()

,SECID,day_date,close,value,volume
0,ABIO,2014-06-09,15.000,22579.4,1500.0
1,ABIO,2014-06-10,14.881,25469.9,1700.0
2,ABIO,2014-06-11,14.999,2999.9,200.0
3,ABIO,2014-06-16,15.000,24522.3,1600.0
4,ABIO,2014-06-17,15.120,738392.0,48900.0


In [7]:
df_market = df_tools.merge(df_candles,on='SECID')
df_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525620 entries, 0 to 525619
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SECID      525620 non-null  object        
 1   BOARDID    525620 non-null  object        
 2   SHORTNAME  525620 non-null  object        
 3   SECNAME    525620 non-null  object        
 4   LOTSIZE    525620 non-null  int64         
 5   SECTYPE    525620 non-null  object        
 6   LISTLEVEL  525620 non-null  int64         
 7   day_date   525620 non-null  datetime64[ns]
 8   close      525620 non-null  float64       
 9   value      525620 non-null  float64       
 10  volume     525620 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 44.1+ MB


### Текстовые данные

In [8]:
df_text['day_date'] = df_text['modified_at'].map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d')) 
df_text = df_text[['day_date','title']].rename(columns={'title':'text'})
df_text = df_text.groupby('day_date',as_index=False)['text'].agg('\n'.join)
df_text.head()

,day_date,text
0,2011-12-17,О конфигурации Фондового рынка Группы ММВБ-РТС...
1,2011-12-29,Об установлении новых условий заключения сдело...
2,2012-01-12,Итоги рынков Группы ММВБ-РТС с 3 по 6 января 2...
3,2012-01-17,Итоги рынков Группы ММВБ-РТС за неделю с 9 по ...
4,2012-01-18,Об установлении новых условий заключения сдело...


## Модель-съедобный вид
Здесь мы превращаем входные данные, в данные которые пойдут непосредственно в регрессоры. А именно, в таблицу со столбцами:
* Целевой инструмент
    * Ид
    * Доп. инфо
    * 30 дней свечей (по отдельному столбцу на день)
* Весь рынок
    * Среднее
    * Дисперсия
    * Квантили
* Текст
    * 30 дней векторов новостей с рынка или с рбк (по отдельному столбцу на день)
* Дата перед предсказываемым днем
    * Год
    * Месяц
    * День месяца
    * День недели

### Обрезка входных данных по 30 дней

In [ ]:
TIME_DELTA = 30
td = df_market['day_date'].max()
fd = td - timedelta(TIME_DELTA)
df_day_delta = pd.DataFrame([i for i in range(TIME_DELTA+1)],columns=['day_delta'])
# df_day_delta

In [10]:
df_market_30_day = df_market[(df_market['day_date'] <= td) & (df_market['day_date'] >= fd)].copy()
df_market_30_day['day_delta'] = (td - df_market_30_day['day_date']).map(lambda x: x.days)
del df_market_30_day['day_date']
df_market_30_day = df_day_delta.merge(df_market_30_day,how='left',on='day_delta')
df_market_30_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5625 entries, 0 to 5624
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   day_delta  5625 non-null   int64  
 1   SECID      5617 non-null   object 
 2   BOARDID    5617 non-null   object 
 3   SHORTNAME  5617 non-null   object 
 4   SECNAME    5617 non-null   object 
 5   LOTSIZE    5617 non-null   float64
 6   SECTYPE    5617 non-null   object 
 7   LISTLEVEL  5617 non-null   float64
 8   close      5617 non-null   float64
 9   value      5617 non-null   float64
 10  volume     5617 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 483.5+ KB


In [11]:
df_text_30_day = df_text[(df_text['day_date'] <= td) & (df_text['day_date'] >= fd)].copy()
df_text_30_day['day_delta'] = (td - df_text_30_day['day_date']).map(lambda x: x.days)
del df_text_30_day['day_date']
df_text_30_day = df_day_delta.merge(df_text_30_day,how='left',on='day_delta')
df_text_30_day[df_text_30_day.isna()] = ''
df_text_30_day

,day_delta,text
0,0,"О внесении изменений в Список ценных бумаг, до..."
1,1,Итоги выпуска биржевых облигаций\nО регистраци...
2,2,"О регистрации выпуска биржевых облигаций, о вк..."
3,3,Дополнительные условия проведения торгов облиг...
4,4,Об обновлении параметров индексов акций\nОб ус...
5,5,
6,6,О начале торгов ценными бумагами &quot;9&quot;...
7,7,О дополнительных условиях предоставления досту...
8,8,Итоги выпуска биржевых облигаций\nО регистраци...
9,9,Об изменении риск-параметров на фондовом рынке...


### Автоматизация

In [51]:
df_text.iloc[-30:]

,day_date,text
3283,2024-11-05,Итоги выпуска биржевых облигаций\nО регистраци...
3284,2024-11-06,"06.11.2024, 18-14 (мск) изменены значения верх..."
3285,2024-11-07,Об установлении коэффициентов free-float ряда ...
3286,2024-11-08,О проведении выкупа облигаций в период с 11 по...
3287,2024-11-11,Об изменении дополнительных условий проведения...
3288,2024-11-12,Об изменении риск-параметров на фондовом рынке...
3289,2024-11-13,О начале торгов ценными бумагами &quot;14&quot...
3290,2024-11-14,Об изменении риск-параметров на фондовом рынке...
3291,2024-11-15,О порядке сбора заявок и заключения сделок при...
3292,2024-11-18,Итоги выпуска биржевых облигаций\nО регистраци...


In [46]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_text['text'])

TfidfVectorizer()

In [ ]:
def transform_ticker(df_market_30_day):
    # ИД и пр. инфо
    df_result = df_market_30_day[[
        'SECID'
        ,'BOARDID'
        ,'SHORTNAME'
        ,'SECNAME'
        ,'LOTSIZE'
        ,'SECTYPE'
        ,'LISTLEVEL'
    ]].drop_duplicates()

    # 30 дней свечей
    df_day_pivot_candles = pd.pivot_table(df_market_30_day,values=['close','value','volume'],columns='day_delta',index=['SECID'],fill_value=-1)
    df_day_pivot_candles.columns = ['_'.join(map(lambda x: str(x),col)) for col in df_day_pivot_candles.columns]
    df_day_pivot_candles.reset_index()

    df_result = df_result.merge(df_day_pivot_candles,on='SECID')
    df_result.columns = ['tools_' + col for col in df_result.columns]
    return df_result

def transform_market_statistic(df_market_30_day):
    # Статистика по рынку
    quantile_25 = lambda x: np.quantile(x,.25)
    quantile_50 = lambda x: np.quantile(x,.5)
    quantile_75 = lambda x: np.quantile(x,.75)
    df_market_statistic = pd.pivot_table(
        df_market_30_day
        ,values=['close','value','volume']
        ,columns='day_delta'
        ,index=[]
        ,aggfunc=[
            'mean'
            ,'std'
            ,quantile_25
            ,quantile_50
            ,quantile_75
        ]
    )
    df_market_statistic = df_market_statistic.unstack().to_frame().T
    df_market_statistic.columns = ['_'.join(map(lambda x: str(x),col)) for col in df_market_statistic.columns]

    df_market_statistic.columns = ['market_' + col for col in df_market_statistic.columns]

    return df_market_statistic

def transform_text(df_text_30_day):
    vectorize_text = vectorizer.transform(df_text_30_day['text'])
    vectorize_text = vectorize_text.reshape((-1,np.prod(vectorize_text.shape)))
    vectorize_text = vectorize_text.toarray()

    df_vectorize_text = pd.DataFrame(vectorize_text)
    df_vectorize_text.columns = ['text_' + str(col) for col in df_vectorize_text.columns]
    
    return df_vectorize_text

def transfrom_one_period(df_market_30_day,df_text_30_day,td):
    df_result = transform_ticker(df_market_30_day)
    
    df_market_statistic = transform_market_statistic(df_market_30_day)
    df_result['key'] = df_market_statistic['key'] = 0 
    df_result = df_result.merge(df_market_statistic,on='key').drop('key',axis=1)

    df_vectorize_text = transform_text(df_text_30_day)
    df_result['key'] = df_vectorize_text['key'] = 0
    df_result = df_result.merge(df_vectorize_text,on='key').drop('key',axis=1)

    df_result['pred_year'] = td.year
    df_result['pred_month'] = td.month
    df_result['pred_day'] = td.day
    df_result['pred_day_of_week'] = td.day_of_week

    return df_result


def transform(df_market,df_text):
    DAY_DELTA = 30
    df_all = df_market.merge(df_text,how='left',on='day_date').reset_index()
    df_all.loc[df_all['text'].isna(),'text'] = ''

    N = len(df_all)
    dfs = []
    for i in range(N - DAY_DELTA + 1):
        df_all_in_period = df_all.iloc[i:i+DAY_DELTA].copy()
        fd = df_all_in_period['day_date'].min()
        td = df_all_in_period['day_date'].max()
        print(i)
        # print(len(df_all_in_period),fd,td)

        df_all_in_period['day_delta'] = (td - df_all_in_period['day_date']).map(lambda x: x.days)

        df_market_in_period = df_all_in_period[[
            'day_delta'
            ,'SECID'
            ,'BOARDID'
            ,'SHORTNAME'
            ,'SECNAME'
            ,'LOTSIZE'
            ,'SECTYPE'
            ,'LISTLEVEL'
            ,'close'
            ,'value'
            ,'volume'
        ]].copy()
        
        df_text_in_period = df_all_in_period[[
            'day_delta'	
            ,'text'
        ]].copy()

        df_transform_period = transfrom_one_period(df_market_in_period,df_text_in_period,td)
        dfs.append(df_transform_period)
    
    return pd.concat(dfs)
    
# transform(df_market.iloc[-31:],df_text[-31:]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 504 entries, 0 to 251
Columns: 357700 entries, tools_SECID to pred_day_of_week
dtypes: float64(357691), int64(4), object(5)
memory usage: 1.3+ GB


# Создание модели